In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from joblib import Parallel, delayed 
from multiprocessing import get_context
import multiprocessing as mp
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()
import warnings
warnings.filterwarnings('ignore')

In [2]:
mp.get_start_method()

'spawn'

In [3]:
pool=get_context('fork').Pool(mp.cpu_count()) 

In [4]:
mp.cpu_count()

8

In [5]:
pool=get_context('fork').Pool(8)

In [6]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

**FUNCION QUE ME COJA LAS 3 PESTAÑAS**

In [7]:
from selenium.common.exceptions import NoSuchElementException

In [8]:
urls=['https://subastas.boe.es/subastas_ava.php?campo%5B0%5D=SUBASTA.ORIGEN&dato%5B0%5D=&campo%5B1%5D=SUBASTA.AUTORIDAD&dato%5B1%5D=&campo%5B2%5D=SUBASTA.ESTADO&dato%5B2%5D=EJ&campo%5B3%5D=BIEN.TIPO&dato%5B3%5D=I&dato%5B4%5D=501&campo%5B5%5D=BIEN.DIRECCION&dato%5B5%5D=&campo%5B6%5D=BIEN.CODPOSTAL&dato%5B6%5D=&campo%5B7%5D=BIEN.LOCALIDAD&dato%5B7%5D=&campo%5B8%5D=BIEN.COD_PROVINCIA&dato%5B8%5D=11&campo%5B9%5D=SUBASTA.POSTURA_MINIMA_MINIMA_LOTES&dato%5B9%5D=&campo%5B10%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_1&dato%5B10%5D=&campo%5B11%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_2&dato%5B11%5D=&campo%5B12%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_3&dato%5B12%5D=&campo%5B13%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_4&dato%5B13%5D=&campo%5B14%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_5&dato%5B14%5D=&campo%5B15%5D=SUBASTA.ID_SUBASTA_BUSCAR&dato%5B15%5D=&campo%5B16%5D=SUBASTA.FECHA_FIN_YMD&dato%5B16%5D%5B0%5D=&dato%5B16%5D%5B1%5D=&campo%5B17%5D=SUBASTA.FECHA_INICIO_YMD&dato%5B17%5D%5B0%5D=&dato%5B17%5D%5B1%5D=&page_hits=50&sort_field%5B0%5D=SUBASTA.FECHA_FIN_YMD&sort_order%5B0%5D=desc&sort_field%5B1%5D=SUBASTA.FECHA_FIN_YMD&sort_order%5B1%5D=asc&sort_field%5B2%5D=SUBASTA.HORA_FIN&sort_order%5B2%5D=asc&accion=Buscar']

      

In [9]:
#le paso los 20 links para haga un loop sobre ellos.     
def boe(url,i):
    
    url=['https://subastas.boe.es/subastas_ava.php?campo%5B0%5D=SUBASTA.ORIGEN&dato%5B0%5D=&campo%5B1%5D=SUBASTA.AUTORIDAD&dato%5B1%5D=&campo%5B2%5D=SUBASTA.ESTADO&dato%5B2%5D=EJ&campo%5B3%5D=BIEN.TIPO&dato%5B3%5D=I&dato%5B4%5D=501&campo%5B5%5D=BIEN.DIRECCION&dato%5B5%5D=&campo%5B6%5D=BIEN.CODPOSTAL&dato%5B6%5D=&campo%5B7%5D=BIEN.LOCALIDAD&dato%5B7%5D=&campo%5B8%5D=BIEN.COD_PROVINCIA&dato%5B8%5D=11&campo%5B9%5D=SUBASTA.POSTURA_MINIMA_MINIMA_LOTES&dato%5B9%5D=&campo%5B10%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_1&dato%5B10%5D=&campo%5B11%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_2&dato%5B11%5D=&campo%5B12%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_3&dato%5B12%5D=&campo%5B13%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_4&dato%5B13%5D=&campo%5B14%5D=SUBASTA.NUM_CUENTA_EXPEDIENTE_5&dato%5B14%5D=&campo%5B15%5D=SUBASTA.ID_SUBASTA_BUSCAR&dato%5B15%5D=&campo%5B16%5D=SUBASTA.FECHA_FIN_YMD&dato%5B16%5D%5B0%5D=&dato%5B16%5D%5B1%5D=&campo%5B17%5D=SUBASTA.FECHA_INICIO_YMD&dato%5B17%5D%5B0%5D=&dato%5B17%5D%5B1%5D=&page_hits=50&sort_field%5B0%5D=SUBASTA.FECHA_FIN_YMD&sort_order%5B0%5D=desc&sort_field%5B1%5D=SUBASTA.FECHA_FIN_YMD&sort_order%5B1%5D=asc&sort_field%5B2%5D=SUBASTA.HORA_FIN&sort_order%5B2%5D=asc&accion=Buscar']
   
    
    driver=webdriver.Chrome(PATH)
    driver.get(url)
    time.sleep(4)

    link=driver.find_element(By.XPATH, f'//*[@id="contenido"]/div[4]/ul/li[{i}]/ul/li/a').click()

    time.sleep(2)

    element= driver.find_element(By.TAG_NAME,'table')

    filas=element.find_elements(By.TAG_NAME,'tr')
    data=[]

    #sacamos los datos de la primera pestaña 
    for f in filas:

        nombre=f.find_elements(By.TAG_NAME, 'th')
        elemen=f.find_elements(By.TAG_NAME, 'td')
        tmp=[]

        try:
            for i,e in zip(nombre,elemen):
                if i.text == 'Identificador':                    
                    tmp.append(e.text)

                if i.text =='Tipo de subasta':
                    tmp.append(e.text)

                if i.text=='Cuenta expediente':
                    tmp.append(e.text)

                if i.text=='Fecha de inicio':
                    tmp.append(e.text)
                if i.text == 'Fecha de conclusión':                    
                    tmp.append(e.text)

                if i.text =='Cantidad reclamada':
                    tmp.append(e.text)

                if i.text=='Anuncio BOE':
                    tmp.append(e.text)

                if i.text=='Valor subasta':
                    tmp.append(e.text)

                if i.text=='Tasación':
                    tmp.append(e.text)

                if i.text=='Puja mínima':
                    tmp.append(e.text)

                if i.text=='Tramos entre pujas':
                    tmp.append(e.text)

                if i.text=='Importe del depósito':
                    tmp.append(e.text)

            data.append(tmp[0])

        except:
            pass

        #ahora sacamos los datos de la segunda pestaña 
    try:
        pestañab=driver.find_element(By.LINK_TEXT, 'Bienes').click()
    except NoSuchElementException:
        pass


    time.sleep(2)

    elementb= driver.find_element(By.TAG_NAME,'table') 
    filasb=elementb.find_elements(By.TAG_NAME,'tr')
    data2=[]

    for f in filasb:

        nombreb=f.find_elements(By.TAG_NAME, 'th')
        elemenb=f.find_elements(By.TAG_NAME, 'td')
        tmp2=[]
        try:
            for i,e in zip(nombreb,elemenb):
                if i.text == 'Dirección':
                    tmp2.append(e.text)

                if i.text =='Código Postal':
                    tmp2.append(e.text)

                if i.text=='Localidad':
                    tmp2.append(e.text)

                if i.text=='Provincia':
                    tmp2.append(e.text)

            data2.append(tmp2[0])

        except:
            pass

  #sacamos los datos de la ultima pestaña
    #value= False

    try:        
        pestañap=driver.find_element(By.LINK_TEXT, 'Pujas').click()
    except NoSuchElementException:
        pass

    time.sleep(2)
    data3=[]

    try:
        contentp = driver.find_element(By.CSS_SELECTOR, 'div.invisible')
    except NoSuchElementException:
        return  data + data2


    filasp=contentp.find_elements(By.TAG_NAME,'p')    


    for f in filasp:
        tmp3=[]
        try:
            for i in filasp:

                    tmp3.append(i.text)

            data3.append(tmp3[0])

        except:
            pass


    driver.quit()

    boe= data + data2 + data3
        

    
    return (boe)  
    

In [13]:
prueb=[]
boe=pd.DataFrame(columns=['identificador', 'tipo_de_subasta', 'cuenta_expediente', 'fecha_de_inicio',
                          'fecha_de_conclusion','cantidad_reclamada','anuncio_BOE','valor_subasta',
                          'tasacion','puja_minima','tramos_entre_pujas','importe_del_deposito','direccion',
                          'codigo_postal','localidad','provincia','puja'])
for url in urls:
    
    lst1=Parallel(n_jobs=-1, verbose=True)(delayed(boe)(url,e) for e in range(1,2))
    df1=pd.DataFrame(lst1)
    #print(df1)
    #prueb.append(lst1)
    boe=boe.append(df1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


TypeError: 'DataFrame' object is not callable

In [ ]:
boe.head()

In [ ]:
boe.to_csv('../data/BOEtotal.csv', index=False, sep=',') 